In [ ]:
import numpy as np
import pandas as pd

def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])        
        result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
    return result

#王老师v2,313ms,0（n2）改成了O(n)，313ms
def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result



In [ ]:

import numpy as np
import pandas as pd


y = np.random.randint(2, size=(5000, 1)) #cython 中无random和dataframe
x = np.random.randint(10, size=(5000, 1))

data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])
#display(data.index,"\n")
#result_1 = target_mean_v1(data, 'y', 'x')
result_2 = target_mean_v2(data, 'y', 'x')
#print(result_2)
#diff = np.linalg.norm(result_1 - result_2)
#print(diff)



In [ ]:
data2=pd.DataFrame([[1,2,3],[11,22,33],[111,222,333]],columns=['col1','col2','col3'])
display(data2)
result=data2.agg(['max','min','mean'])
result.index

In [ ]:
#v3改进用nparray数组替代dataframe在循环中的调用,80.3ms

def target_mean_v3(data:pd.DataFrame,y_name:str,x_name:str):
  rows=data.shape[0]
  result=np.zeros(rows)
  value_dict=dict()
  count_dict=dict()
  x_series=data[x_name]
  y_series=data[y_name]
  for i in range(rows):
    x_single=x_series[i]
    y_single=y_series[i]
    if x_single in value_dict:
      value_dict[x_single]+=y_single
      count_dict[x_single]+=1
    else:
      value_dict[x_single]=y_single
      count_dict[x_single]=1
  for i in range(rows):
    x_single=x_series[i]
    y_single=y_series[i]
    result[i]=(value_dict[x_single]-y_single)/(count_dict[x_single]-1)
  return result


In [ ]:
#v4改进用nparray数组替代dataframe在循环中的调用,结果8.83ms

def target_mean_v4(data:pd.DataFrame,y_name:str,x_name:str):
  rows=data.shape[0]
  result=np.zeros(rows)
  value_dict=dict()
  count_dict=dict()
  x_series=np.array(data[x_name])
  y_series=np.array(data[y_name])
  for i in range(rows):
    x_single=x_series[i]
    y_single=y_series[i]
    if x_single in value_dict:
      value_dict[x_single]+=y_single
      count_dict[x_single]+=1
    else:
      value_dict[x_single]=y_single
      count_dict[x_single]=1
  for i in range(rows):
    x_single=x_series[i]
    y_single=y_series[i]
    result[i]=(value_dict[x_single]-y_single)/(count_dict[x_single]-1)
  return result


In [ ]:
#v4上cython
%%cython -a -ano

cimport cython
import numpy as np
cimport numpy as cnp
import pandas as pd
from libcpp.algorithm cimport sort as stdsort
from libcpp.vector cimport vector

def target_mean_v5(data,cnp.str y_name,cnp.str x_name):
  cdef:
   int rows=data.shape[0]
   cnp.ndarray[float] result=np.zeros(rows,dtype=float)
   dict value_dict={}
   dict count_dict={}
   cnp.ndarray[float] x_series=np.array(data[x_name])
   cnp.ndarray[float] y_series=np.array(data[y_name])
  for i from 0<=i<rows:
    x_single=x_series[i]
    y_single=y_series[i]
    if x_single in value_dict:
      value_dict[x_single]+=y_single
      count_dict[x_single]+=1
    else:
      value_dict[x_single]=y_single
      count_dict[x_single]=1
  for i from 0<=i<rows:
    x_single=x_series[i]
    y_single=y_series[i]
    result[i]=(value_dict[x_single]-y_single)/(count_dict[x_single]-1)
  return result

In [ ]:
from line_profiler import LineProfiler
import numpy as np
import pandas as pd

profiler=LineProfiler(target_mean_v2)
profiler.enable()
target_mean_v3(data,'y','x')
profiler.disable()
profiler.print_stats

%load_ext line_profiler
%lprun -f target_mean_v5 target_mean_v5(data,'y','x')



In [ ]:
#import timeit
%%timeit
target_mean_v5(data,'y','x')